## NOTES
**V1**
solved ratio +/-1 issues; except for test 18...
<br/>
making a copy to preserve this version
<br/>
**V2**
DOUBLES ARE SENSITIVE; adjust for triples/multi to recognize low abundance barcodes in more complex samples
<br/>
making copy (v3) to preserve this version
<br/>
**V3**
MULTI WORKS ( added cutoff for previous threshold before calc new thr, post clean)
<br/>
making a copy (v4) to clean up version 
### V4 (this version)
2016-05-01:  from 2016-04-12-ratio_thresholding, renamed from threshold_new_20160412-v4.ipynb

In [1]:
%matplotlib inline
import pandas as pd
import numpy as np
import regex
import os, sys


In [2]:
EXP_NAME = '2016-07-27-20-pantaplates-9815-9615'
FILTERED_FILE = '../output/filtered_2016-07-27-20-pantaplates-9815-9615.csv'
MIN_READS = 1000
GROUP_BY = ['monkey','tube','sample_type']

In [64]:
def calculate_percent_molecs(df):
    df.sort_values(by=['molecs'],ascending=False,inplace=True)
    total = float(df.molecs.values.sum()) / 100.
    df.loc[:,'percent_molecs'] = df.molecs.apply(lambda x: x/total)
    df.reset_index(inplace=True,drop=True)
    return df

In [41]:
def load_data(filtered_file):
    # loads excel file (all tabs)
    data = pd.read_csv(filtered_file)
    data.reset_index(inplace=True,drop=True)
    
    # clean up no qtags, adds per_molecs column
    data = data.loc[data.qtag!='None']
    data = data.groupby(['monkey','tube','sample_type']).apply(calculate_percent_molecs)
    
    # formatting
    columns = ['monkey','tube','qtag','gtag','molecs','reads','sample_type']
    data = data[columns]
    data.sort_values(by=['monkey','tube','sample_type','molecs'], inplace=True, ascending=[True,True,False,False])
    data.reset_index(inplace=True,drop=True)
    data.loc[:,'monkey'] = data.monkey.apply(lambda x: int(x))
    
#     print data.head()
#     print data.dtypes
#     print data.loc[data]
#     print data
    data.loc[:,'tube'] = data.tube.apply(lambda x: x if x[:1].upper()=='P' else int(x.split('.')[0])  )
    return data


In [5]:
def calculate_threshold_simple(y):
    yarray = np.concatenate([ [y[0]], y, [0] ])
    return np.argmin(np.diff(yarray,n=2))+1

In [83]:
# STEP 2: CALCULATE THRESHOLD via. CONCAVITY
def calculate_threshold_few_complex(y):
    
    def rolling_window(arr):
        shape = arr.shape[:-1] + (arr.shape[-1]-1, 2)
        strides = arr.strides + (arr.strides[-1],)
        windows = np.lib.stride_tricks.as_strided(arr, 
                              shape=shape, strides=strides)
        return windows
    
    def first_d_gen(windows):
        for w in windows:
            yield float(w[1]-w[0])/w[0]
    def second_d_gen(windows):
        for w in windows:
            yield w[1]-w[0]         
    def derive(arr):
        windows = rolling_window(y)
        der = np.fromiter(derivative_gen(windows), 
                          np.float, count=len(windows))
        return der
    
    yarray = np.concatenate([ [y[0]], y, [1] ])
    windows_y = rolling_window(yarray)
    first_d = np.fromiter(first_d_gen(windows_y), np.float, count=len(windows_y))
    windows_1 = rolling_window(first_d)
    second_d = np.fromiter(second_d_gen(windows_1), np.float, count=len(windows_1))
    
    return np.argmin(second_d)+1

In [7]:
# STEP 3: CLEAN BY ELIMINATING ONE-OFFS
def eliminate_oneoffs(group, threshold, thr_i, pad=True):
    
    group.loc[:,'delete'] = group.molecs.apply(lambda _: False)
    counter = 0
    # add capability to check other parameters, i.e. qtag
    for majorI, marow in group.loc[group.percent_molecs > threshold].iterrows():
        majorG, majorQ, majorDel = marow[['gtag','qtag','delete']]
        if majorDel == False:
            motif = "(%s){s<=1}"%majorG
            subgroup = group[counter+1:].loc[(group.delete==False)]
            for minorI, mirow in subgroup.iterrows():
                minorG = mirow['gtag']
                query = regex.search(motif,minorG)
                if query:
                    group.loc[minorI,'delete'] = True
        counter+=1
        
    output = group.loc[(group.delete==False)]
    if len(group.loc[group.delete==True])> 0 and pad==True:
        maxdelete = group.loc[group.delete==True].molecs.idxmax()
        output = group.loc[(group.delete==False) | (group.index==maxdelete)]
        output.loc[:,'delete'] = False    
    return output
            

In [82]:
# THRESHOLD A SAMPLE/INDEX/DF GROUP
def threshold_group(group, reps_remaining, threshold, threshold_i):
    
    group = calculate_percent_molecs(group).reset_index(drop=True)
    cleaned = eliminate_oneoffs(group,threshold, threshold_i)
    
    if threshold > 0:
        cleaned = cleaned.loc[cleaned.percent_molecs >= threshold]
    
    
    initial_thr = calculate_threshold_few_complex(cleaned.molecs.values)+1
    thr_i = min(initial_thr, len(cleaned)-1)
    new_threshold = group.percent_molecs.values[thr_i]

    # max out reps
    if reps_remaining <= 0:
        print 'Maxed out reps. Skipping group.'
        return 

    # if reached steady state
    elif new_threshold == threshold:
        prepass = cleaned.loc[cleaned.molecs >= group.molecs.values[thr_i]]
        prepass.reset_index(drop=True, inplace=True)
        prepass2 = eliminate_oneoffs(prepass, -1, len(prepass), pad=False)
        prepass.reset_index(drop=True, inplace=True)

        thr_i = min(calculate_threshold_few_complex(prepass2.molecs.values)+1, len(prepass2)-1)
        threshold = prepass2.percent_molecs.values[thr_i]
        passed = prepass2.loc[prepass2.percent_molecs >= threshold]
        passed = calculate_percent_molecs(passed)
        return passed, threshold, thr_i
    
    # recursively clean and re-threshold
    else:
        return threshold_group(cleaned, reps_remaining-1, new_threshold, thr_i)

In [86]:
data

,monkey,tube,qtag,gtag,molecs,reads,sample_type
0,9615,1,q23,TCGATGT,169,169,727.0
1,9615,1,q24,CGTGGTG,1,1,727.0
2,9615,11,q24,CGTGGTG,161,161,727.0
3,9615,12,q24,CGTGGTG,224,224,727.0
4,9615,12,q24,CGTGTTG,1,1,727.0
5,9615,13,q24,AGTGGTT,615,616,727.0
6,9615,13,q19,GAGGTAG,1,1,727.0
7,9615,13,q24,AGTCGTT,1,1,727.0
8,9615,13,q24,AGTGATT,1,1,727.0
9,9615,13,q24,AGTGGGT,1,1,727.0


In [99]:

# RUN THRESHOLDING FOR ALL SAMPLES IN DICT
def run_threshold(d):
    tvals = {}
    tis = {}
    passed = {}
    counter = 1
    
    datadict = d if type(d)==type(dict()) else dict(((i,g) for i,g in d.groupby(GROUP_BY)))
    for idx in datadict:
        msg = "Index %d of %d (%s): "%(counter,len(datadict),idx)
        group = datadict[idx]
        group = group.loc[(group.qtag!='None') & (group.gtag!='None') & (group.molecs>50)]
        if len(group) > 0 and max(group.reads.values) > MIN_READS:
            result = threshold_group(group, 20, -1, len(group)+2)
            if result:
                passed[idx], tvals[idx], tis[idx] = result
                msg += "Thresholded.\n" 
            else: msg += "Skipped.\n"
        else: msg+= "Skipped.\n"
        sys.stdout.write(msg)
        sys.stdout.flush()
        counter += 1

    try:
        passeddf = pd.concat(passed.values())
        passeddf.loc[:,'monkey'] = passeddf.monkey.apply(lambda x: int(x))
#         passeddf.loc[:,'tube'] = passeddf.tube.apply(lambda x: int(x))
        passeddf.sort_values(by=GROUP_BY+['percent_molecs'],ascending=[True,True,False,False],inplace=True)
        passeddf.reset_index(inplace=True, drop=True)
        passeddf.drop(['delete'], axis=1, inplace=True)
        
    except Exception, e:
        print 'EXCEPTION', e
        passeddf = passed
    return passeddf, tvals, tis

In [100]:
data = load_data(FILTERED_FILE)

In [101]:
passed, threshold_values, threshold_i = run_threshold(data)

Index 1 of 53 ((9815, 9, 720.0)): Thresholded.


C:\Users\vwl698\Anaconda2\lib\site-packages\ipykernel\__main__.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  from ipykernel import kernelapp as app


Index 2 of 53 ((9615, 18, 727.0)): Thresholded.
Index 3 of 53 ((9815, 13, 720.0)): Thresholded.
Index 4 of 53 ((9815, 40, 720.0)): Skipped.
Index 5 of 53 ((9615, 24, 727.0)): Thresholded.
Index 6 of 53 ((9615, 13, 727.0)): Skipped.
Index 7 of 53 ((9615, 6, 727.0)): Skipped.
Index 8 of 53 ((9815, 25, 720.0)): Thresholded.
Index 9 of 53 ((9615, 11, 727.0)): Skipped.
Index 10 of 53 ((9615, 12, 727.0)): Skipped.
Index 11 of 53 ((9815, 19, 720.0)): Thresholded.
Index 12 of 53 ((9815, 33, 720.0)): Thresholded.
Index 13 of 53 ((9815, 30, 720.0)): Thresholded.
Index 14 of 53 ((9815, 28, 720.0)): Thresholded.
Index 15 of 53 ((9815, 5, 720.0)): Thresholded.
Index 16 of 53 ((9815, 23, 727.0)): Thresholded.
Index 17 of 53 ((9615, 22, 727.0)): Skipped.
Index 18 of 53 ((9815, 34, 727.0)): Skipped.
Index 19 of 53 ((9615, 27, 727.0)): Skipped.
Index 20 of 53 ((9815, 36, 720.0)): Thresholded.
Index 21 of 53 ((17914, 'p1', 727.0)): Skipped.
Index 22 of 53 ((9615, 1, 727.0)): Skipped.
Index 23 of 53 ((96

In [102]:
counts = passed.groupby(GROUP_BY).agg(len)['gtag']
counts.name='barcodes'
counts = pd.DataFrame(counts)
counts.reset_index(inplace=True)

In [103]:
passed.loc[:,'passed_threshold'] = True

In [104]:
passed.head()

,monkey,tube,qtag,gtag,molecs,reads,sample_type,percent_molecs,passed_threshold
0,9615,16,q19,GAGGTAG,1116,1120,727.0,100.0,True
1,9615,18,q24,CGTGGTG,1146,1149,727.0,100.0,True
2,9615,19,q24,CGTGGTG,1821,1829,727.0,100.0,True
3,9615,24,q24,CGTGGTG,1124,1129,727.0,100.0,True
4,9615,26,q24,CGTCGTT,1128,1133,727.0,100.0,True


In [107]:
merged = pd.merge(data,passed,on=['monkey','tube','sample_type','qtag','gtag'], how='outer')
merged.drop(['molecs_y','reads_y','percent_molecs'],axis=1, inplace=True)
merged.rename(columns={'molecs_x':'molecs','reads_x':'reads'},inplace=True)
merged.passed_threshold = merged.passed_threshold.fillna(False)

In [109]:
merged.to_csv('../output/thresholded-%s-merged.csv'%EXP_NAME)

In [110]:
pd.DataFrame(counts).to_csv('../output/thresholded-%s-counts.csv'%EXP_NAME)


In [111]:
passed.to_csv('../output/thresholded-%s-passed.csv'%EXP_NAME)